# Creating Fresh Data Base

In [7]:
import psycopg2

def get_create_table_queries(conn):
    create_table_queries = []
    try:
        # Create a cursor object
        cur = conn.cursor()

        # Get the list of all schemas excluding temporary and system schemas
        schema_sql = """
        SELECT schema_name 
        FROM information_schema.schemata
        WHERE schema_name NOT LIKE 'pg_temp_%'
        AND schema_name NOT LIKE 'pg_toast%'
        AND schema_name NOT LIKE 'information%'
        AND schema_name NOT LIKE 'pg_catalog%'
        """
        cur.execute(schema_sql)
        schemas = cur.fetchall()

        # Get the CREATE TABLE statements for all tables in each schema
        for schema in schemas:
            schema_name = schema[0]
            table_sql = f"""
            SELECT table_name 
            FROM information_schema.tables
            WHERE table_schema = '{schema_name}'
            """
            cur.execute(table_sql)
            tables = cur.fetchall()

            for table in tables:
                table_name = table[0]
                create_sql = f"""
                SELECT 'CREATE TABLE ' || quote_ident('{schema_name}') || '.' || quote_ident('{table_name}') || E'\n(\n' ||
                array_to_string(
                    array_agg(
                        '    ' || quote_ident(column_name) || ' ' || data_type ||
                        coalesce('(' || character_maximum_length || ')', '') ||
                        case when is_nullable = 'NO' then ' NOT NULL' else '' end
                    ), E',\n'
                ) || E'\n);\n'
                FROM information_schema.columns
                WHERE table_schema = '{schema_name}' AND table_name = '{table_name}'
                GROUP BY table_schema, table_name
                """
                cur.execute(create_sql)
                create_table_query = cur.fetchone()[0]
                create_table_queries.append(create_table_query)

        # Close the cursor
        cur.close()

    except Exception as e:
        print(f"Error: {e}")

    return create_table_queries

def save_to_sql_file(filename, queries):
    with open(filename, 'w') as f:
        for query in queries:
            f.write(query + '\n\n')


conn = psycopg2.connect(
    host="192.168.2.54",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# create the schemas


create_table_queries = get_create_table_queries(conn)
save_to_sql_file("create_tables.sql", create_table_queries)


# Close the connection
conn.close()

# Run this cell to create all the tables and schemas

In [2]:
import psycopg2
# connect to local host db named BravisaDB
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# run the create_table.sql file 
with open("create_tables.sql", 'r') as f:
    create_table_sql = f.read()
    cur = conn.cursor()
    cur.execute(create_table_sql)
    conn.commit()
    cur.close()
    

# UPPER casing all the files in OHLC folder 

In [17]:
import os

cwd = os.getcwd()

ohlc = os.path.join(cwd, "app\\OHLCFiles")

# for files starting with cm and ending with .csv rename then to upper()
for file in os.listdir(ohlc):
    if file.startswith("cm") and file.endswith(".csv"):
        os.rename(os.path.join(ohlc, file), os.path.join(ohlc, file.upper()))

## Monthly missing BackgroundInfo

In [7]:
import psycopg2
import pandas as pd
import datetime

conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

current_date = '2024-01-04'
start_date__of_the_month = datetime.datetime.strptime(current_date, '%Y-%m-%d').replace(day=1).strftime('%Y-%m-%d')
end_date_of_the_month = pd.to_datetime(current_date).to_period('M').end_time.strftime('%Y-%m-%d')

print(start_date__of_the_month, end_date_of_the_month)

btt_list_sql = """SELECT * FROM public."BTTList" """
# where "BTTDate" >= '{}' and "BTTDate" <= '{}'""".format(start_date__of_the_month, end_date_of_the_month)
btt_list = pd.read_sql_query(btt_list_sql, conn)

background_info_sql = """SELECT * FROM public."BackgroundInfo" """
background_info = pd.read_sql_query(background_info_sql, conn)

#for distinct companycodes, check both tables have them else report the missing companycodes from backgroundinfo
missing_companycodes = []
for companycode in btt_list['CompanyCode'].unique():
    if companycode not in background_info['CompanyCode'].unique():
        missing_companycodes.append(companycode)
print(len(missing_companycodes))
print(missing_companycodes)

# for distinct ISIN codes in bttlist, check if they are present in backgroundinfo and report the missing ISIN codes
missing_isin = []
for isin in btt_list['ISIN'].unique():
    if isin not in background_info['ISINCode'].unique():
        missing_isin.append(isin)
print(len(missing_isin))
print(missing_isin)
print()

2024-01-01 2024-01-31
5
[11050093.0, 15620019.0, 16070141.0, 16590031.0, 17090024.0]
100
['34-TRIAL-INE451A01017 100', 'INE066A01013', '34-TRIAL-INE371A01025 100', 'INE545A01016', 'INE999A01015', 'INE765D01014', 'INE570A01014', 'INE802C01017', 'INE180C01026', 'INE927D01028', 'INE918I01018', 'INE466L01020', 'INE602A01023', 'INE597I01010', 'INE254N01018', 'INE982B01019', 'INE775B01025', 'INE036B01022', 'INE040A01026', 'INE858B01011', 'INE476A01014', 'INE239A01016', 'INE528G01027', 'INE00VM01010', 'INE200M01013', 'INE797F01012', 'INE121A01016', 'INE229C01013', 'INE239C01012', 'INE00WC01019', 'INE513A01014', 'INE962Y01013', 'INE821I01014', 'INE461D01010', 'INE116A01024', 'INE150B01021', 'INE410B01029', 'INE884A01019', 'INE863B01011', 'INE486A01013', 'INE035D01012', 'INE663A01017', 'INE183A01016', 'INE081A01012', 'INE335A01012', 'INE324A01024', 'INE451D01011', 'INE702C01019', 'INE385C01013', 'INE943C01027', 'INE825A01012', 'INE572A01028', 'INE064C01014', 'INE571A01020', 'INE930H01023', 'INE

In [5]:
import psycopg2
import pandas as pd
import datetime

conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

current_date = '2024-01-05'
start_date__of_the_month = datetime.datetime.strptime(current_date, '%Y-%m-%d').replace(day=1).strftime('%Y-%m-%d')
end_date_of_the_month = pd.to_datetime(current_date).to_period('M').end_time.strftime('%Y-%m-%d')

print(start_date__of_the_month, end_date_of_the_month)

btt_list_sql = """SELECT * FROM public."BTTList" """
                #   where "BTTDate" >= '{}' and "BTTDate" <= '{}'""".format(start_date__of_the_month, end_date_of_the_month)
btt_list = pd.read_sql_query(btt_list_sql, conn)

ShareHolding_sql = """SELECT * FROM public."ShareHolding" """ 
ShareHolding = pd.read_sql_query(ShareHolding_sql, conn)

# get missing companycodes that are present in bttlist but not in ShareHolding
missing_companycodes = []
for companycode in btt_list['CompanyCode'].unique():
    if companycode not in ShareHolding['CompanyCode'].unique():
        missing_companycodes.append(companycode)
print(len(missing_companycodes))
print(missing_companycodes)




2024-01-01 2024-01-31
13
[14094822.0, 14094977.0, 14094980.0, 17200004.0, 17200007.0, 17200009.0, 17200012.0, 14095554.0, 17200016.0, 17200019.0, 17200023.0, 15150124.0, 15210746.0]


In [15]:
# program to compamre two csvfiles
import pandas as pd
import datetime
from utils.db_helper import DB_Helper

conn = DB_Helper().db_connect()
cur = conn.cursor()

# '2024-01-05'
curr_date = datetime.datetime.strptime('2024-01-05', '%Y-%m-%d')

file1 = pd.read_csv('ohlc_new.csv')
file2 = pd.read_csv('old_ohlc.csv')

# sort them based on company code
file1 = file1.sort_values(by='CompanyCode')
file2 = file2.sort_values(by='CompanyCode')

# print null rows in file1 where the subset of colmns are CompanyCode, NSECode, BSECode, ISIN
print(len(file1[file1[['CompanyCode', 'NSECode', 'BSECode', 'ISIN']].isnull().any(axis=1)]))
print(len(file2[file2[['CompanyCode', 'NSECode', 'BSECode', 'ISIN']].isnull().any(axis=1)]))

BTT_back = datetime.date(curr_date.year, curr_date.month, 1).strftime("%Y-%m-%d")
next_month = curr_date.month + 1 if curr_date.month + 1 <= 12 else 1
next_year = curr_date.year if curr_date.month + 1 <= 12 else curr_date.year + 1
BTT_next = datetime.date(next_year, next_month, 1).strftime("%Y-%m-%d")

# Fetch BTT list data for the current month
btt_sql = """
SELECT *
FROM "BTTList"
WHERE "BTTDate" >= %s AND "BTTDate" < %s
"""
bttlist = pd.read_sql_query(btt_sql, con=conn, params=(BTT_back, BTT_next))
    
# print the missing company codes in file1 that are present in bttlist
missing_companycodes = []
for companycode in bttlist['CompanyCode'].unique():
    if companycode not in file1['CompanyCode'].unique():
        missing_companycodes.append(companycode)

print(len(missing_companycodes))
print(missing_companycodes)

2111
2128
2
[14094980.0, 17200012.0]


In [ ]:
# program to compamre two csvfiles
import pandas as pd
import datetime
from utils.db_helper import DB_Helper

conn = DB_Helper().db_connect()
cur = conn.cursor()



In [6]:
from utils.db_helper import DB_Helper
import pandas as pd
conn = DB_Helper().db_connect()
cur = conn.cursor()

# get shareholding columns
shareholding_sql = """SELECT * FROM public."ShareHolding" """
shareholding = pd.read_sql_query(shareholding_sql, conn)

print(shareholding.columns)

C:\Users\dsram\AppData\Local\Temp\ipykernel_6344\453760155.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  shareholding = pd.read_sql_query(shareholding_sql, conn)


Index(['CompanyCode', 'SHPDate', 'Capital', 'FaceValue', 'NoOfShares',
       'Promoters', 'Directors', 'SubsidiaryCompanies', 'OtherCompanies',
       'ICICI', 'UTI', 'IDBI', 'GenInsuranceComp', 'LifeInsuranceComp',
       'StateFinCorps', 'InduFinCorpIndia', 'ForeignNRI',
       'ForeignCollaborators', 'ForeignOCB', 'ForeignOthers',
       'ForeignInstitutions', 'ForeignIndustries', 'StateGovt', 'CentralGovt',
       'GovtCompanies', 'GovtOthers', 'Others', 'NBanksMutualFunds',
       'HoldingCompanies', 'GeneralPublic', 'Employees',
       'FinancialInstitutions', 'ForeignPromoter', 'GDR',
       'PersonActingInConcert', 'Total', 'ModifiedDate'],
      dtype='object')


# Back-up Script

In [3]:
import os
import subprocess
from utils.db_helper import DB_Helper

# Connect to the database
conn = DB_Helper().db_connect()
cur = conn.cursor()

# Create backup directory if it does not exist
backup_dir = 'backup'
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)

# Backup the database using pg_dump
backup_file = os.path.join(backup_dir, 'backup_05_02.sql')
pg_dump_command = f'pg_dump -U postgres -h localhost -p 5432 -d BravisaDB -F c -b -v -f {backup_file}'

# Set the PGPASSWORD environment variable for the PostgreSQL user password
os.environ['PGPASSWORD'] = 'edsols'

# Execute the pg_dump command and capture the output
process = subprocess.Popen(pg_dump_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command was successful
if process.returncode == 0:
    print(f"Database backup successful. Backup file created at {backup_file}")
else:
    print(f"Database backup failed with exit code {process.returncode}")
    print(f"Error message: {stderr.decode()}")

# Clean up the environment variable
del os.environ['PGPASSWORD']


Database backup successful. Backup file created at backup\backup_05_02.sql


In [ ]:
import os
import subprocess
from utils.db_helper import DB_Helper

# Connect to the database
conn = DB_Helper().db_connect()
cur = conn.cursor()

# Create backup directory if it does not exist
backup_dir = 'backup'
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)

# Backup the database using pg_dump
backup_file = os.path.join(backup_dir, 'backup.sql')
pg_dump_command = f'pg_dump -U postgres -h localhost -p 5432 -d BravisaDB -F c -b -v -f {backup_file}'

# Set the PGPASSWORD environment variable for the PostgreSQL user password
os.environ['PGPASSWORD'] = 'edsols'

# Execute the pg_dump command and capture the output
process = subprocess.Popen(pg_dump_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command was successful
if process.returncode == 0:
    print(f"Database backup successful. Backup file created at {backup_file}")
else:
    print(f"Database backup failed with exit code {process.returncode}")
    print(f"Error message: {stderr.decode()}")

# Clean up the environment variable
del os.environ['PGPASSWORD']


In [ ]:
# restore the database using pg_restore
restore_file = os.path.join(backup_dir, 'backup.sql')
pg_restore_command = f'pg_restore -U postgres -h localhost -p 5432 -d BravisaDB -c -v -F c -d {restore_file}'
os.environ['PGPASSWORD'] = 'edsols'

# Execute the pg_restore command and capture the output
process = subprocess.Popen(pg_restore_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command was successful
if process.returncode == 0:
    print(f"Database restore successful. Database restored from {restore_file}")
else:
    print(f"Database restore failed with exit code {process.returncode}")
    print(f"Error message: {stderr.decode()}")
    
# Clean up the environment variable
del os.environ['PGPASSWORD']


# Recovery script

In [ ]:
import os
import shutil

def create_recovery_conf(archive_dir, recovery_target_time, data_dir):
    recovery_conf_path = os.path.join(data_dir, 'recovery.conf')

    recovery_conf_content = f"""
restore_command = 'copy {archive_dir}\\%f %p'
recovery_target_time = '{recovery_target_time}'
"""

    with open(recovery_conf_path, 'w') as file:
        file.write(recovery_conf_content)
    
    print(f'recovery.conf created at {recovery_conf_path}')

def restart_postgresql():
    os.system('net stop postgresql-x64-12')
    os.system('net start postgresql-x64-12')
    print('PostgreSQL service restarted')

if __name__ == "__main__":
    # Specify the paths and recovery target time
    archive_directory = r'C:\postgres_archive'
    recovery_time = '2024-12-11 16:00:00'
    postgresql_data_directory = r'C:\Program Files\PostgreSQL\<version>\data'

    # Create recovery.conf file
    create_recovery_conf(archive_directory, recovery_time, postgresql_data_directory)

    # Restart PostgreSQL service
    restart_postgresql()


In [9]:
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()

cur = conn.cursor()

last_insert_date_sql = """SELECT MAX("log_date") FROM "logs"."insert" """
last_report_generation_date_sql = """SELECT MAX("log_date") FROM "logs"."report_generation" """

last_insert_date = pd.read_sql_query(last_insert_date_sql, conn)
last_report_generation_date = pd.read_sql_query(last_report_generation_date_sql, conn)

print(last_insert_date.values[0][0])
print(last_report_generation_date.values[0][0])


2024-11-29
2024-11-28


C:\Users\dsram\AppData\Local\Temp\ipykernel_9196\1454160463.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_insert_date = pd.read_sql_query(last_insert_date_sql, conn)
C:\Users\dsram\AppData\Local\Temp\ipykernel_9196\1454160463.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_report_generation_date = pd.read_sql_query(last_report_generation_date_sql, conn)


# Gathering all results from FB and missing results 
from what I've seen the Quarterly Results files have ModifiedDate column with that I'm trying to order them all from the missing and FB files and creating a new QuarterlyResults directory to use to insert results before running EPS from first

In [42]:
import os
import zipfile
import shutil
missing_dir = 'D:\\FB Files Backup\\'

ignore_list = ['2020', '2021', '2022', '2023', '2024', 'OHLCFiles', 'ALLFBFiles', 'BTT Data (FB Files Full Backup)', 'drive-download', 'OHLC files']

extract_dir = os.path.join(os.getcwd(), 'missing')

for folder in os.listdir(missing_dir):
    if folder not in ignore_list:
        print(folder)
        for file in os.listdir(os.path.join(missing_dir, folder)):
            if file.endswith('.zip'):
                print(file)
                with zipfile.ZipFile(os.path.join(missing_dir, folder, file), 'r') as zip_ref:
                    zip_ref.extractall(os.path.join(extract_dir, file.split('.zip')[0]))
                    print('Extracted', file)
            if file.endswith('.csv'):
                print(file)
                shutil.copy(os.path.join(missing_dir, folder, file), os.path.join(extract_dir, file))
                print('Copied', file)

Missing Result
Apr to June-21.zip
Extracted Apr to June-21.zip
Bravisa_Consolidated_HQNResultsData.zip
Extracted Bravisa_Consolidated_HQNResultsData.zip
Bravisa_FinanceData.zip
Extracted Bravisa_FinanceData.zip
Bravisa_HQNData_12082021.zip
Extracted Bravisa_HQNData_12082021.zip
Bravisa_HQNData_12082021_New.zip
Extracted Bravisa_HQNData_12082021_New.zip
Bravisa_HQNResultsData.zip
Extracted Bravisa_HQNResultsData.zip
Bravisa_HQNResultsData_18022024.zip
Extracted Bravisa_HQNResultsData_18022024.zip
Bravisa_HQNResults_07102021.zip
Extracted Bravisa_HQNResults_07102021.zip
Bravisa_NAVData_01042021.zip
Extracted Bravisa_NAVData_01042021.zip
Bravisa_ResultsData (1).zip
Extracted Bravisa_ResultsData (1).zip
Bravisa_ResultsData.zip
Extracted Bravisa_ResultsData.zip
Bravisa_ResultsData_30302022 (1).zip
Extracted Bravisa_ResultsData_30302022 (1).zip
Bravisa_Sep2022_MissingHQResults.zip
Extracted Bravisa_Sep2022_MissingHQResults.zip
ConsolidatedHalfyearlyResults.csv
Copied ConsolidatedHalfyearlyRe

# folders and files not in FBfiles dir

In [25]:
# recusrively check for .zip files in the missing directory that from the reference list reference_list = ['FB0101202077'] then check for FB0101202101.zip in the missing directory
# and check if they are present in the missing directory and vice versa
# if they are not present in the missing directory, print the missing files
# if they are not present in the reference directory, print the missing files

import os
import zipfile

reference_dir = os.path.join(os.getcwd(), 'FBFiles')
reference_list = os.listdir(reference_dir)
print(reference_list)

missing_dir = 'D:\\FB Files Backup\\'

ignore_list = ['2020', '2021', '2022', '2023', '2024', 'OHLCFiles']

for root, dirs, files in os.walk(missing_dir):
    for file in files:
        if file.endswith('.zip'):
            if file[:-4] not in reference_list:
                print(file, "not in reference list")
        else:
            if not file.endswith(('.BTC')):
                print(file, "not a zip file in missing directory")
                    
                    

['FB0101202101', 'FB0101202102', 'FB0101202103', 'FB0101202201', 'FB0101202202', 'FB0101202203', 'FB0101202401', 'FB0101202402', 'FB0101202403', 'FB0102202101', 'FB0102202102', 'FB0102202103', 'FB0102202201', 'FB0102202202', 'FB0102202203', 'FB0102202301', 'FB0102202302', 'FB0102202303', 'FB0102202401', 'FB0102202402', 'FB0102202403', 'FB0103202101', 'FB0103202102', 'FB0103202103', 'FB0103202201', 'FB0103202202', 'FB0103202203', 'FB0103202301', 'FB0103202302', 'FB0103202303', 'FB0103202401', 'FB0103202402', 'FB0103202403', 'FB0104202101', 'FB0104202102', 'FB0104202103', 'FB0104202201', 'FB0104202202', 'FB0104202203', 'FB0104202301', 'FB0104202302', 'FB0104202303', 'FB0104202401', 'FB0104202402', 'FB0104202403', 'FB0105202101', 'FB0105202102', 'FB0105202103', 'FB0105202401', 'FB0105202402', 'FB0105202403', 'FB0106202101', 'FB0106202102', 'FB0106202103', 'FB0106202201', 'FB0106202202', 'FB0106202203', 'FB0106202301', 'FB0106202302', 'FB0106202303', 'FB0106202401', 'FB0106202402', 'FB0106

## creating a collective missing QuarterlyResults

In [ ]:
import os
import pandas as pd

missing_dir = os.path.join(os.getcwd(), 'missing')

QuarterlyResults = pd.DataFrame()

for f in os.listdir(missing_dir):
    if f.endswith('.csv'):
        if f == 'QuarterlyResults.csv':
            df = pd.read_csv(os.path.join(missing_dir, f))
            # copy the data to the QuarterlyResults dataframe
            QuarterlyResults = pd.concat([QuarterlyResults, df])
    else: 
        for files in os.listdir(os.path.join(missing_dir, f)):
            print(f, files)
            if files.endswith('.csv'):
                if files == 'QuarterlyResults.csv':
                    df = pd.read_csv(os.path.join(missing_dir, f, files))
                    QuarterlyResults = pd.concat([QuarterlyResults, df])
                    
# save the QuarterlyResults dataframe to a csv file
QuarterlyResults.to_csv('missing/MissingQuarterlyResults.csv', index=False)

Apr to June-21 April21
Apr to June-21 June-21
Apr to June-21 May-21
Bravisa FullData1997 to Sept 2016 ._AnnualGeneralMeeting.csv
Bravisa FullData1997 to Sept 2016 ._Auditors.csv
Bravisa FullData1997 to Sept 2016 ._Auditors.xlsx
Bravisa FullData1997 to Sept 2016 ._BackgroundInfo.csv
Bravisa FullData1997 to Sept 2016 ._BackgroundInfo.xlsx
Bravisa FullData1997 to Sept 2016 ._Bankers.csv
Bravisa FullData1997 to Sept 2016 ._BoardMeetings.csv
Bravisa FullData1997 to Sept 2016 ._Bonus.csv
Bravisa FullData1997 to Sept 2016 ._CapitalStructure.csv
Bravisa FullData1997 to Sept 2016 ._CompanyMaster.csv
Bravisa FullData1997 to Sept 2016 ._CompanyNameChange.csv
Bravisa FullData1997 to Sept 2016 ._ConsolidatedHalfyearlyResults.csv
Bravisa FullData1997 to Sept 2016 ._ConsolidatedNinemonthsResults.csv
Bravisa FullData1997 to Sept 2016 ._ConsolidatedQuarterlyResults.csv
Bravisa FullData1997 to Sept 2016 ._CurrentData.csv
Bravisa FullData1997 to Sept 2016 ._Dividend.csv
Bravisa FullData1997 to Sept 2016 

In [ ]:
import os
import pandas as pd

missing_dir = os.path.join(os.getcwd(), 'missing')

ConsolidatedQuarterlyResults = pd.DataFrame()

for f in os.listdir(missing_dir):
    if f.endswith('.csv'):
        if f == 'ConsolidatedQuarterlyResults.csv':
            df = pd.read_csv(os.path.join(missing_dir, f))
            # copy the data to the QuarterlyResults dataframe
            ConsolidatedQuarterlyResults = pd.concat([QuarterlyResults, df])
    else: 
        for files in os.listdir(os.path.join(missing_dir, f)):
            print(f, files)
            if files.endswith('.csv'):
                if files == 'ConsolidatedQuarterlyResults.csv':
                    df = pd.read_csv(os.path.join(missing_dir, f, files))
                    QuarterlyResults = pd.concat([ConsolidatedQuarterlyResults, df])
                    
# save the QuarterlyResults dataframe to a csv file
ConsolidatedQuarterlyResults.to_csv('missing/MissingConsolidatedQuarterlyResults.csv', index=False)

Apr to June-21 April21
Apr to June-21 June-21
Apr to June-21 May-21
Bravisa FullData1997 to Sept 2016 ._AnnualGeneralMeeting.csv
Bravisa FullData1997 to Sept 2016 ._Auditors.csv
Bravisa FullData1997 to Sept 2016 ._Auditors.xlsx
Bravisa FullData1997 to Sept 2016 ._BackgroundInfo.csv
Bravisa FullData1997 to Sept 2016 ._BackgroundInfo.xlsx
Bravisa FullData1997 to Sept 2016 ._Bankers.csv
Bravisa FullData1997 to Sept 2016 ._BoardMeetings.csv
Bravisa FullData1997 to Sept 2016 ._Bonus.csv
Bravisa FullData1997 to Sept 2016 ._CapitalStructure.csv
Bravisa FullData1997 to Sept 2016 ._CompanyMaster.csv
Bravisa FullData1997 to Sept 2016 ._CompanyNameChange.csv
Bravisa FullData1997 to Sept 2016 ._ConsolidatedHalfyearlyResults.csv
Bravisa FullData1997 to Sept 2016 ._ConsolidatedNinemonthsResults.csv
Bravisa FullData1997 to Sept 2016 ._ConsolidatedQuarterlyResults.csv
Bravisa FullData1997 to Sept 2016 ._CurrentData.csv
Bravisa FullData1997 to Sept 2016 ._Dividend.csv
Bravisa FullData1997 to Sept 2016 

# EPS rerun with Missing Results

In [2]:
import os
import pandas as pd
from reports.EPS import EPS
from utils.db_helper import DB_Helper
from config import FB_FOLDER
from datetime import datetime, timedelta, date
import warnings
warnings.filterwarnings("ignore")

missing_dir = os.path.join(os.getcwd(), 'missing')

file_path = FB_FOLDER

# column_renames = {
# 	'IncomeOnInvestment': 'IncomeOnInvestments',
# 	'ResearchAndDevlopmentExpenses': 'ResearchAndDevelopmentExpenses',
# 	'PriorYearAdj': 'PrioryearAdj',
# 	'AggregateofNonPromotoNoOfShares': 'AggregateOfNonPromotoNoOfShares',
# 	'AggregateofNonPromotoHoldingPercent': 'AggregateOfNonPromotoHoldingPercent',
# 	'CapitalAdequacyBaseII': 'CapitalAdequacyBasell'
# }

MissingQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingQuarterlyResults.csv'))
# MissingQuarterlyResults.rename(columns=column_renames, inplace=True)
MissingQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingQuarterlyResults['ModifiedDate'])

# MissingQuarterlyResults.rename(columns=column_renames, inplace=True)
# print(MissingQuarterlyResults['ModifiedDate'])
MissingConsolidatedQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingConsolidatedQuarterlyResults.csv'))
# MissingConsolidatedQuarterlyResults.rename(columns=column_renames, inplace=True)
MissingConsolidatedQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingConsolidatedQuarterlyResults['ModifiedDate'])
# MissingConsolidatedQuarterlyResults.rename(columns=column_renames, inplace=True)
# print(MissingConsolidatedQuarterlyResults['ModifiedDate'])
from datetime import datetime

def get_closest_quarter(target):
    if not isinstance(target, datetime):
        raise ValueError("target must be a datetime object")
    
    year = target.year
    quarters = [
        datetime(year, 3, 31),
        datetime(year, 6, 30),
        datetime(year, 9, 30),
        datetime(year, 12, 31),
        datetime(year - 1, 12, 31)
    ]
    
    closest_quarter = min(quarters, key=lambda q: abs(q - target))
    return closest_quarter

def delete_EPS(conn,cur, curr_date):
    try:
        year_ending = EPS().get_closest_quarter(curr_date)
        print(f"Deleting EPS data for date: {curr_date} and year ending: {year_ending}")
        
        eps_sql = """DELETE FROM "Reports"."EPS"
                    WHERE "EPSDate" = %s """
        cur.execute(eps_sql, (curr_date,)) 
        conn.commit()
        print("Deleted EPS data")
        
        STANDALONE_EPS_sql = """ DELETE FROM "Reports"."STANDALONE_EPS"
                    WHERE "EPSDate" = %s"""
        cur.execute(STANDALONE_EPS_sql, (curr_date,))
        conn.commit()
        print("Deleted STANDALONE_EPS data")
        
        Consolidated_EPS_sql = """ DELETE FROM "Reports"."Consolidated_EPS"
                    WHERE "EPSDate" = %s"""
        cur.execute(Consolidated_EPS_sql, (curr_date,))
        conn.commit()
        print("Deleted Consolidated_EPS data")
        
        QaurterlyEPS_sql = """ DELETE FROM public."QuarterlyEPS"
                    WHERE "YearEnding" = %s"""
        cur.execute(QaurterlyEPS_sql, (year_ending,))
        conn.commit()
        print("Deleted QuarterlyEPS data")
        
        ConsolidatedQuarterlyEPS_sql = """ DELETE FROM public."ConsolidatedQuarterlyEPS"
                    WHERE "YearEnding" = %s"""
        cur.execute(ConsolidatedQuarterlyEPS_sql, (year_ending,))
        conn.commit()
        print("Deleted ConsolidatedQuarterlyEPS data")
        
        TTMsql = """ DELETE FROM public."TTM"
                    WHERE "YearEnding" = %s"""
        cur.execute(TTMsql, (year_ending,))
        conn.commit()
        print("Deleted TTM data")
        
        ConsolidatedTTMsql = """ DELETE FROM public."ConsolidatedTTM"
                    WHERE "YearEnding" = %s"""
        cur.execute(ConsolidatedTTMsql, (year_ending,))
        conn.commit()
        print("Deleted ConsolidatedTTM data")
        
    except Exception as e:
        print(f"Error deleting EPS data: {e}")
        import traceback; traceback.print_exc()


def insert_quarterlyresults(conn, cur,fbname ):
		""" Insert the Quarterly Results data into database

		Operation:
			Set the path for csv file, and fetch the data from QuarterlyResults.csv file.
			delete the data from QuarterlyEPS, TTM and QuarterlyResults tables based on 
			key column CompanyCode and YearEnding and export executed data into
			'QuarterlyResultsExport.csv' file and insert into QuarterlyResults Table.
		"""
		date =fbname[6:10] + '-' + fbname[4:6] + '-' + fbname[2:4]
  		#convert date to date format
		date = pd.to_datetime(date)
		fb_name = fbname
		fb_csv_path = os.path.join(file_path, fb_name +'\\') 
		
		fb_csv_file = fb_csv_path + 'QuarterlyResults.csv'
		file_to_check = fb_csv_file
				
		if(fbname == 'intermediate_insert'):
			
			file_to_check = 'C:\\Users\\dsram\\BravisaLocalDeploy\\BravisaFiles\\MissingData'+ '\\' + 'QuarterlyResults.csv'
			fb_csv_file = file_to_check

		if os.path.isfile(file_to_check):
			try:
				concat_MissingQuarterlyResults = MissingQuarterlyResults[MissingQuarterlyResults['ModifiedDate'].dt.date == date.date()]
				# print("concat_MissingQuarterlyResults: ", concat_MissingQuarterlyResults)
				table = pd.read_csv(fb_csv_file, engine='python')				
				print("table before concat: ", table.shape)
				table = pd.concat([table, concat_MissingQuarterlyResults])
				print("table after concat: ", table.shape)
    
				#Update Logic - Deletes based on key columns 
				print("Executing delete logic", flush = True)
				table_to_delete = table.groupby(['CompanyCode', 'YearEnding'], as_index=False).count()
				'''self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
							"\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
				self.logfile.flush()'''
				for index,row in table_to_delete.iterrows():
					# self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
					# 		"\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
					# self.logfile.flush()

					sql_eps = 'DELETE FROM public."QuarterlyEPS" WHERE "CompanyCode" =\'' +str(row['CompanyCode'])  + '\' AND "YearEnding"=\'' + str(row['YearEnding']) + '\';'
					sql_ttm = 'DELETE FROM public."TTM" WHERE "CompanyCode" =\'' +str(row['CompanyCode'])  + '\' AND "YearEnding"=\'' + str(row['YearEnding']) + '\';'
					sql = 'DELETE FROM public."QuarterlyResults" WHERE "CompanyCode" =\'' +str(row['CompanyCode'])  + '\' AND "YearEnding"=\'' + str(row['YearEnding']) + '\';'
					
					cur.execute(sql_eps)
					cur.execute(sql_ttm)
					cur.execute(sql)

					conn.commit()
				

				print("Inserting Data into QuarterlyResults", flush = True)
				exportfilename = "QuarterlyResultsExport.csv"
				exportfile = open(exportfilename,"w",encoding='utf-8')
				table.to_csv(exportfile, header=True, index=False, lineterminator='\r')
				exportfile.close()
				
				copy_sql = """
					COPY "public"."QuarterlyResults" FROM stdin WITH CSV HEADER
					DELIMITER as ','
					"""
				with open(exportfilename, 'r') as f:
					cur.copy_expert(sql=copy_sql, file=f)
					conn.commit()
					f.close()
				os.remove(exportfilename)
			except:
				print("Error in insert_quarterlyresults()", flush = True)
				import traceback; traceback.print_exc();
				conn.rollback()

		else:
			print("File not found: " + fb_csv_file, flush = True)


			
def insert_consolidatedquarterlyresults(conn, cur, fbname):
		""" Insert the Consolidated Quarterly Results data into database

		Operation:
			Set the path for csv file, and fetch the data from ConsolidatedQuarterlyResults.csv file.
			delete the data based on key column, CompanyCode and YearEnding
			export executed data into 'ConsolidatedQuarterlyResultsExport.csv' file 
			and insert into ConsolidatedQuarterlyResults Table. 
		"""
		# extract date from fbname FB0101202101 - 01-01-2021
		date =fbname[6:10] + '-' + fbname[4:6] + '-' + fbname[2:4]
		#convert date to date format
		date = pd.to_datetime(date)

  
		# print("consolidated insert Date: ", date)
		fb_name = fbname 
		fb_csv_path = os.path.join(file_path, fb_name +'\\')	
		fb_csv_file = fb_csv_path + 'ConsolidatedQuarterlyResults.csv'
		
		file_to_check = fb_csv_file
		if(fbname == 'intermediate_insert'):
			
			file_to_check = 'C:\\Users\\dsram\\BravisaLocalDeploy\\BravisaFiles\\MissingData'+ '\\' + 'ConsolidatedQuarterlyResults.csv'
			fb_csv_file = file_to_check

		if os.path.isfile(file_to_check):
			try:
				# subset of MissingConsolidatedQuarterlyResults based on date
				concat_MissingConsolidatedQuarterlyResults = MissingConsolidatedQuarterlyResults[MissingConsolidatedQuarterlyResults['ModifiedDate'].dt.date == date.date()]
				# print("concat_MissingConsolidatedQuarterlyResults: ", concat_MissingConsolidatedQuarterlyResults)
				table = pd.read_csv(fb_csv_file, engine='python')
				print("table before concat: ", table.shape)
				table = pd.concat([table, concat_MissingConsolidatedQuarterlyResults])
				print("table after concat: ", table.shape)
				#Update Logic - Deletes based on key columns 
				print("Executing delete logic", flush = True)
				table_to_delete = table.groupby(['CompanyCode', 'YearEnding'], as_index=False).count()
				'''self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
							"\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
				self.logfile.flush()'''
				for index,row in table_to_delete.iterrows():
					# self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
					# 		"\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
					# self.logfile.flush()
					sql = 'DELETE FROM public."ConsolidatedQuarterlyResults" WHERE "CompanyCode"=\'' +str(row['CompanyCode'])+ '\' AND "YearEnding"=\'' + str(row['YearEnding'])+'\';'   
					sql_eps = 'DELETE FROM public."ConsolidatedQuarterlyEPS" WHERE "CompanyCode" =\'' +str(row['CompanyCode'])  + '\' AND "YearEnding"=\'' + str(row['YearEnding']) + '\';'
					sql_ttm = 'DELETE FROM public."ConsolidatedTTM" WHERE "CompanyCode" =\'' +str(row['CompanyCode'])  + '\' AND "YearEnding"=\'' + str(row['YearEnding']) + '\';'
					
					cur.execute(sql)
					cur.execute(sql_eps)
					cur.execute(sql_ttm)
					conn.commit()
				

				print("Inserting Data into ConsolidatedQuarterlyResults", flush = True)
				exportfilename = "ConsolidatedQuarterlyResultsExport.csv"
				exportfile = open(exportfilename,"w",encoding='utf-8')
				table.to_csv(exportfile, header=True, index=False, lineterminator='\r')
				exportfile.close()
				
				copy_sql = """
					COPY "public"."ConsolidatedQuarterlyResults" FROM stdin WITH CSV HEADER
					DELIMITER as ','
					"""
				with open(exportfilename, 'r') as f:
					cur.copy_expert(sql=copy_sql, file=f)
					conn.commit()
					f.close()
				os.remove(exportfilename)
			
			except:
				print("Error in insert_consolidatedquarterlyresults()", flush = True)
				import traceback; traceback.print_exc();
				conn.rollback()

		else:
			print("File not found here: " + fb_csv_file, flush = True)

	


try:
	conn = DB_Helper().db_connect()
	cur = conn.cursor()
	date_range = pd.date_range(start='2021-09-24',end='2021-09-24')
	start_date = date_range[0]
	end_date = date_range[-1]
	# delelte EPS data for that day
	date_format = "%d%m%Y"
	current_date = start_date
	# delete_EPS(conn, cur, start_date, end_date)
	while current_date <= end_date:
		if current_date.weekday() == 0:  # Monday
			prev_date = current_date - timedelta(days=2)  # Saturday
		else:
			prev_date = current_date - timedelta(days=1)  # Previous day
		delete_EPS(conn, cur, current_date)
		print(current_date)
		try:
			insert_quarterlyresults(conn, cur, "FB" + current_date.strftime(date_format) + "01")
			insert_quarterlyresults(conn, cur, "FB" + prev_date.strftime(date_format) + "03")
			insert_quarterlyresults(conn, cur, "FB" + current_date.strftime(date_format) + "02")
			print("Inserted: QuarterlyResults for : ", current_date)
			insert_consolidatedquarterlyresults(conn, cur, "FB" + current_date.strftime(date_format) + "01")
			insert_consolidatedquarterlyresults(conn, cur, "FB" + prev_date.strftime(date_format) + "03")
			insert_consolidatedquarterlyresults(conn, cur, "FB" + current_date.strftime(date_format) + "02")
			print("Inserted: ConsolidatedQuarterlyResults for : ", current_date)
		except Exception as e:
			print(f"Error processing date {current_date}: {e}")
			conn.rollback()
			break
		try:
			EPS().Generate_Daily_Report(current_date.date(),conn,cur)  
			print("Generated EPS report for: ", current_date)
		except FileNotFoundError:
			print(" there is no file for the date: ", current_date)
	
		current_date += timedelta(days=1)

except Exception as e:
	print(f"Error connecting to the database: {e}")
	import traceback; traceback.print_exc();
finally:
	if cur:
		cur.close()
	if conn:
		conn.close()



Deleting EPS data for date: 2021-09-24 00:00:00 and year ending: 2021-06-30
Deleted EPS data
Deleted STANDALONE_EPS data
Deleted Consolidated_EPS data
Deleted QuarterlyEPS data
Deleted ConsolidatedQuarterlyEPS data
Deleted TTM data
Deleted ConsolidatedTTM data
2021-09-24 00:00:00
table before concat:  (0, 61)
table after concat:  (2, 61)
Executing delete logic
Inserting Data into QuarterlyResults
table before concat:  (2, 61)
table after concat:  (6, 61)
Executing delete logic
Inserting Data into QuarterlyResults
table before concat:  (0, 61)
table after concat:  (2, 61)
Executing delete logic
Inserting Data into QuarterlyResults
Inserted: QuarterlyResults for :  2021-09-24 00:00:00
table before concat:  (0, 65)
table after concat:  (0, 65)
Executing delete logic
Inserting Data into ConsolidatedQuarterlyResults
table before concat:  (1, 65)
table after concat:  (2, 65)
Executing delete logic
Inserting Data into ConsolidatedQuarterlyResults
table before concat:  (0, 65)
table after conc

# inserting missing results till 2018 december


In [ ]:
# inserting missing results till 2018 december
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

missing_dir = os.path.join(os.getcwd(), 'missing')

date = datetime.strptime('2018-12-01', '%Y-%m-%d')

MissingQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingQuarterlyResults.csv'))
MissingQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingQuarterlyResults['ModifiedDate']) 
# subset of MissingQuarterlyResulst but ModifiedDate less than 2018-12-01
MissingQuarterlyResults = MissingQuarterlyResults[MissingQuarterlyResults['ModifiedDate'].dt.date <= date.date()]
MissingConsolidatedQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingConsolidatedQuarterlyResults.csv'))
MissingConsolidatedQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingConsolidatedQuarterlyResults['ModifiedDate'])
MissingConsolidatedQuarterlyResults = MissingConsolidatedQuarterlyResults[MissingConsolidatedQuarterlyResults['ModifiedDate'].dt.date <= date.date()]

print(MissingQuarterlyResults.shape)
print(MissingConsolidatedQuarterlyResults.shape)

# fetch QuarterlyResults till 2018 december
fetch_quarterlyresults_sql = """SELECT * FROM public."QuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
fetch_consolidatedquarterlyresults_sql = """SELECT * FROM public."ConsolidatedQuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""

quarterlyresults = pd.read_sql_query(fetch_quarterlyresults_sql, conn)
consolidatedquarterlyresults = pd.read_sql_query(fetch_consolidatedquarterlyresults_sql, conn)
print("from db")    
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

# concatenate the missing results with the fetched results
quarterlyresults = pd.concat([quarterlyresults, MissingQuarterlyResults])
consolidatedquarterlyresults = pd.concat([consolidatedquarterlyresults, MissingConsolidatedQuarterlyResults])
print("after concatenation")
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

#sort by modified date
quarterlyresults = quarterlyresults.sort_values(by='ModifiedDate')
consolidatedquarterlyresults = consolidatedquarterlyresults.sort_values(by='ModifiedDate')
#drop duplicates
quarterlyresults = quarterlyresults.drop_duplicates(['CompanyCode', 'YearEnding'], keep='last')
consolidatedquarterlyresults = consolidatedquarterlyresults.drop_duplicates(['CompanyCode', 'YearEnding'], keep='last')
print("after dropping duplicates")
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

# delete the data from the tables
delete_quarterlyresults_sql = """DELETE FROM public."QuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
delete_consolidatedquarterlyresults_sql = """DELETE FROM public."ConsolidatedQuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
try:
    cur.execute(delete_quarterlyresults_sql)
    cur.execute(delete_consolidatedquarterlyresults_sql)
    conn.commit()
    print("Deleted data till 2018-12-01")
except Exception as e:
    print(f"Error deleting data: {e}")
    import traceback; traceback.print_exc()
    conn.rollback()



(206387, 61)
(0, 65)


C:\Users\dsram\AppData\Local\Temp\ipykernel_16124\2128147327.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quarterlyresults = pd.read_sql_query(fetch_quarterlyresults_sql, conn)
C:\Users\dsram\AppData\Local\Temp\ipykernel_16124\2128147327.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  consolidatedquarterlyresults = pd.read_sql_query(fetch_consolidatedquarterlyresults_sql, conn)


from db
(144484, 61)
(1327, 65)
after concatenation
(350871, 68)
(1327, 71)
after dropping duplicates
(292388, 68)
(1327, 71)
Deleted data till 2018-12-01
Error inserting data: extra data after last expected column
CONTEXT:  COPY QuarterlyResults, line 2: "14080001.0,1996-06-30,3,1,2645828000.0,0.0,2645828000.0,0.0,0.0,0.0,0.0,1872000.0,0.0,0.0,0.0,0.0,0...."



Traceback (most recent call last):
  File "C:\Users\dsram\AppData\Local\Temp\ipykernel_16124\2128147327.py", line 76, in <module>
    cur.copy_expert(sql=copy_sql, file=f)
psycopg2.errors.BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY QuarterlyResults, line 2: "14080001.0,1996-06-30,3,1,2645828000.0,0.0,2645828000.0,0.0,0.0,0.0,0.0,1872000.0,0.0,0.0,0.0,0.0,0...."



# Initial irs_index_mapping table

In [ ]:
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn =db.db_connect()
cur = conn.cursor()


INDEX_MAPPING = {
                        'INDUSTRY-Automobiles': 'AUTO-I',
                        'INDUSTRY-Automobile Components': 'AUTOCOMP-I',
                        'INDUSTRY-Specialty Retail': 'SPECRETAI-I',
                        'INDUSTRY-Consumer Staples Distribution & Retail': 'CONSTAPDIST-I',
                        'INDUSTRY-Broadline Retail': 'BROADRETAI-I',
                        'INDUSTRY-Food Products': 'FOODPROD-I',
                        'INDUSTRY-Beverages': 'BEVERAGE-I',
                        'INDUSTRY-Building Products': 'BUILDPROD-I',
                        'INDUSTRY-Construction & Engineering': 'CONSENGINE-I',
                        'INDUSTRY-Chemicals': 'CHEMICAL-I',
                        'INDUSTRY-Textiles, Apparel & Luxury Goods': 'TEXAPPLUX-I',
                        'INDUSTRY-Containers & Packaging': 'CONTAINPKG-I',
                        'INDUSTRY-Oil, Gas & Consumable Fuels': 'ONGCONFUEL-I',
                        'INDUSTRY-Household Products': 'HOUSEPROD-I',
                        'INDUSTRY-Personal Care Products': 'PERCARPROD-I',
                        'INDUSTRY-Pharmaceuticals': 'PHARMAC-I',
                        'INDUSTRY-Health Care Equipment & Supplies': 'HLTCAEQSUPP-I',
                        'INDUSTRY-Tobacco': 'TOBACCO-I',
                        'INDUSTRY-Household Durables': 'HOUSEDUR-I',
                        'INDUSTRY-Technology Hardware, Storage & Peripherals': 'TECHARDSOFTPER-I',
                        'INDUSTRY-Software': 'SOFTWARE-I',
                        'INDUSTRY-Electronic Equipment, Instruments & Components': 'ELEEQINSCMP-I',
                        'INDUSTRY-Commercial Services & Supplies': 'COMSERVSUP-I',
                        'INDUSTRY-Electrical Equipment': 'ELECEQUIP-I',
                        'INDUSTRY-IT Services': 'ITSERV-I',
                        'INDUSTRY-Machinery': 'MACHINERY-I',
                        'INDUSTRY-Insurance': 'INSURAN-I',
                        'INDUSTRY-Banks': 'BANKS-I',
                        'INDUSTRY-Financial Services': 'FINSERV-I',
                        'INDUSTRY-Capital Markets': 'CAPIMKT-I',
                        'INDUSTRY-Metals & Mining': 'METAMINE-I',
                        'INDUSTRY-Energy Equipment & Services': 'ENEQUSERV-I',
                        'INDUSTRY-Electric Utilities': 'ELECUTIL-I',
                        'INDUSTRY-Diversified Telecommunication Services': 'DIVTELECOMSERV-I',
                        'INDUSTRY-Independent Power and Renewable Electricity Producers': 'INDPOWRENEL-I',
                        'INDUSTRY-Paper & Forest Products': 'PAPFORPROD-I',
                        'INDUSTRY-Health Care Providers & Services': 'HLTCAPROSERV-I',
                        'INDUSTRY-Marine Transportation': 'MARITRA-I',
                        'INDUSTRY-Ground Transportation': 'GRNDTRAN-SI',
                        'INDUSTRY-Air Freight & Logistics': 'AIRFRELOGIS-I',
                        'INDUSTRY-Hotels, Restaurants & Leisure': 'HOTRESLEIS-I',
                        'INDUSTRY-Transportation Infrastructure': 'TRANSINFRA-I',
                        'INDUSTRY-Interactive Media & Services': 'INTMEDSERV-I',
                        'INDUSTRY-Diversified Consumer Services': 'DIVERCONSERV-I',
                        'INDUSTRY-Real Estate Management & Development': 'RESMGMDEV-I',
                        'INDUSTRY-Entertainment': 'ENTERTAIN-I',
                        'INDUSTRY-Retail REITs': 'RETIALREIT-I',
                        'INDUSTRY-Media': 'MEDIA-I',
                        'INDUSTRY-Gas Utilities': 'GASUTIL-I',
                        'INDUSTRY-Leisure Products': 'LEISPROD-I',
                        'SUBINDUSTRY-Auto - LCVs/HCVs': 'AUTLCVHCV-SI',
                        'SUBINDUSTRY-Auto - Cars & Jeeps': 'AUTCARJEE-SI',
                        'SUBINDUSTRY-Motorcycle Manufacturers': 'MOTORMFG-SI',
                        'SUBINDUSTRY-Automotive Parts & Equipment': 'AUTPEQUIP-SI',
                        'SUBINDUSTRY-Tires & Rubber': 'TIRERUB-SI',
                        'SUBINDUSTRY-Carbon Black': 'CARBLCK-SI',
                        'SUBINDUSTRY-Other Specialty Retail': 'OTHSPECRETL-SI',
                        'SUBINDUSTRY-Retail - Departmental Stores': 'RETDEPTSTO-SI',
                        'SUBINDUSTRY-Retail - Apparel': 'RETAPPAREL-SI',
                        'SUBINDUSTRY-Retail - Apparel/Accessories': 'RETAPPARACC-SI',
                        'SUBINDUSTRY-Trading': 'TRADING-SI',
                        'SUBINDUSTRY-Plantations': 'PLANTATIO-SI',
                        'SUBINDUSTRY-Distillers & Vintners': 'DISTILVINTN-SI',
                        'SUBINDUSTRY-Aqua & Horticulture': 'AQUAHORTIC-SI',
                        'SUBINDUSTRY-Edible Oils': 'EDIBLEOIL-SI',
                        'SUBINDUSTRY-Agricultural Products & Services': 'AGRIPROSER-SI',
                        'SUBINDUSTRY-Soft Drinks & Non-alcoholic Beverages': 'SODRINALCBEV-SI',
                        'SUBINDUSTRY-Packaged Foods & Meats': 'PKGFOODMEAT-SI',
                        'SUBINDUSTRY-Sugar': 'SUGAR-SI',
                        'SUBINDUSTRY-Cement & Products': 'CEMPROD-SI',
                        'SUBINDUSTRY-Tiles & Granites': 'TILEGRANI-SI',
                        'SUBINDUSTRY-Decoratives & Laminates': 'DECOLAMIN-SI',
                        'SUBINDUSTRY-Paints': 'PAINTS-SI',
                        'SUBINDUSTRY-Building Products': 'BUILDPROD-SI',
                        'SUBINDUSTRY-Construction & Engineering': 'CONSENGINE-SI',
                        'SUBINDUSTRY-Commodity Chemicals': 'COMMOCHEM-SI',
                        'SUBINDUSTRY-Specialty Chemicals': 'SPECHEM-SI',
                        'SUBINDUSTRY-Fertilizers & Agricultural Chemicals': 'FERAGRICHM-SI',
                        'SUBINDUSTRY-Diversified Chemicals': 'DIVERCHEM-SI',
                        'SUBINDUSTRY-Textiles': 'TEXTILES-SI',
                        'SUBINDUSTRY-Paper & Plastic Packaging Products & Materials': 'PPLPGPROMAT-SI',
                        'SUBINDUSTRY-Oil & Gas Refining & Marketing': 'ONGREFMKT-SI',
                        'SUBINDUSTRY-Household Products': 'HOUSEPROD-SI',
                        'SUBINDUSTRY-Personal Care Products': 'PERCARPROD-SI',
                        'SUBINDUSTRY-Pharmaceuticals': 'PHARMAC-SI',
                        'SUBINDUSTRY-Apparel, Accessories & Luxury Goods': 'APPACCLUX-SI',
                        'SUBINDUSTRY-Health Care Supplies': 'HLTCARESUPP-SI',
                        'SUBINDUSTRY-Tobacco': 'TOBACCO-SI',
                        'SUBINDUSTRY-Household Appliances': 'HOUSEAPPLI-SI',
                        'SUBINDUSTRY-Technology Hardware, Storage & Peripherals': 'TECHARDSOFTPER-SI',
                        'SUBINDUSTRY-Systems Software': 'SYSOFT-SI',
                        'SUBINDUSTRY-Electronic Equipment & Instruments': 'ELECEQUINS-SI',
                        'SUBINDUSTRY-Office Services & Supplies': 'OFFSERVSUPP-SI',
                        'SUBINDUSTRY-Consumer Electronics': 'CONSELEC-SI',
                        'SUBINDUSTRY-Electrical Components & Equipment': 'ELECOMEQU-SI',
                        'SUBINDUSTRY-Health Care Equipment': 'HLTCAREQU-SI',
                        'SUBINDUSTRY-Application Software': 'APPLICSOFT-SI',
                        'SUBINDUSTRY-IT Consulting & Other Services': 'ITCONSOTHSV-SI',
                        'SUBINDUSTRY-Research & Consulting Services': 'RESCONSERV-SI',
                        'SUBINDUSTRY-Industrial Machinery & Supplies & Components': 'INDMCHSUPCOM-SI',
                        'SUBINDUSTRY-Finance - Life Insurance': 'FINLIFINS-SI',
                        'SUBINDUSTRY-Private Banks': 'PVTBNK-SI',
                        'SUBINDUSTRY-Finance - Term Lending': 'FINTERLEND-SI',
                        'SUBINDUSTRY-Finance - Mutual Funds': 'FINMF-SI',
                        'SUBINDUSTRY-Investment Trusts': 'INVTRUST-SI',
                        'SUBINDUSTRY-Finance - Housing': 'FINHSG-SI',
                        'SUBINDUSTRY-Finance & Investments': 'FININVTS-SI',
                        'SUBINDUSTRY-PSU Banks': 'PSUBNK-SI',
                        'SUBINDUSTRY-Finance - Non Life Insurance': 'FINNONLIFINS-SI',
                        'SUBINDUSTRY-Reinsurance': 'REINSURE-SI',
                        'SUBINDUSTRY-Financial Exchanges & Data': 'FINEXCHDATA-SI',
                        'SUBINDUSTRY-Bearings': 'BEARINGS-SI',
                        'SUBINDUSTRY-Ferro Alloys': 'FERALLO-SI',
                        'SUBINDUSTRY-Fasteners': 'FASTENERS-SI',
                        'SUBINDUSTRY-Industrial Gases': 'INDUSGAS-SI',
                        'SUBINDUSTRY-Metal, Glass & Plastic Containers': 'METGLAPLACON-SI',
                        'SUBINDUSTRY-Aluminum': 'ALUMINIUM-SI',
                        'SUBINDUSTRY-Precious Metals & Minerals': 'PRECMETMIN-SI',
                        'SUBINDUSTRY-Diversified Metals & Mining': 'DIVERMETMIN-SI',
                        'SUBINDUSTRY-Oil & Gas Drilling': 'ONGDRILL-SI',
                        'SUBINDUSTRY-Electric Utilities': 'ELECUTILI-SI',
                        'SUBINDUSTRY-Telecommunications - Equipment': 'TELECOMEQU-SI',
                        'SUBINDUSTRY-Heavy Electrical Equipment': 'HVYELECEQUIP-SI',
                        'SUBINDUSTRY-Renewable Electricity': 'RENEWELEC-SI',
                        'SUBINDUSTRY-Copper': 'COPPER-SI',
                        'SUBINDUSTRY-Integrated Telecommunication Services': 'INTGTELCOSVC-SI',
                        'SUBINDUSTRY-Infrastructure - General': 'INFRAGEN-SI',
                        'SUBINDUSTRY-Steel': 'STEEL-SI',
                        'SUBINDUSTRY-Paper Products': 'PAPERPROD-SI1',
                        'SUBINDUSTRY-Livestock - Hatcheries/Poultry': 'LIVHATCHPOUL-SI',
                        'SUBINDUSTRY-Health Care Facilities': 'HLTCAREFACIL-SI',
                        'SUBINDUSTRY-Marine Transportation': 'MARITRA-SI',
                        'SUBINDUSTRY-Transport - Road': 'TRAROAD-SI',
                        'SUBINDUSTRY-Transport - Air': 'TRAAIR-SI',
                        'SUBINDUSTRY-Hotels, Resorts & Cruise Lines': 'HOTRESCRUIS-SI',
                        'SUBINDUSTRY-Marine Ports & Services': 'MARIPORSV-SI',
                        'SUBINDUSTRY-Diversified Support Services': 'DIVERSUPSER-SI',
                        'SUBINDUSTRY-Interactive Media & Services': 'INTMEDSERV-SI',
                        'SUBINDUSTRY-Fire Protection Equipment': 'FIRPROTEQU-SI',
                        'SUBINDUSTRY-LPG Bottling/Distribution': 'LPGBOTDIST-SI',
                        'SUBINDUSTRY-Commercial Printing': 'COMMERPRINT-SI',
                        'SUBINDUSTRY-Agricultural & Farm Machinery': 'AGRIFARMCH-SI',
                        'SUBINDUSTRY-Diversified Financial Services': 'DIVERFINSER-SI',
                        'SUBINDUSTRY-E-Commerce - Retail': 'ECOMRET-SI',
                        'SUBINDUSTRY-Education Services': 'EDUSERV-SI',
                        'SUBINDUSTRY-Real Estate Development': 'REALSTATDEV-SI',
                        'SUBINDUSTRY-Waste Management': 'WASTEMAN-SI',
                        'SUBINDUSTRY-Multi-Sector Holdings': 'MULTSECTHLD-SI',
                        'SUBINDUSTRY-Fintech': 'FINTEC-SI',
                        'SUBINDUSTRY-Health Care Services': 'HLTCARESERV-SI',
                        'SUBINDUSTRY-Digital Entertainment': 'DIGIENTER-SI',
                        'SUBINDUSTRY-Leisure Facilities': 'LEISFACIL-SI',
                        'SUBINDUSTRY-Dairy': 'DAIRY-SI',
                        'SUBINDUSTRY-Marine Foods': 'MARIFOOD-SI',
                        'SUBINDUSTRY-Road Infrastructure': 'ROADINFRA-SI',
                        'SUBINDUSTRY-Retail REITs': 'RETAILREIT-SI',
                        'SUBINDUSTRY-Specialized Finance': 'SPECIFIN-SI',
                        'SUBINDUSTRY-Railway Wagons and Wans': 'RAILWAGWAN-SI',
                        'SUBINDUSTRY-Footwear': 'FOOTWEAR-SI',
                        'SUBINDUSTRY-Advertising': 'ADVERT-SI',
                        'SUBINDUSTRY-Construction Machinery & Heavy Transportation Equipment': 'CONMCHVYTRN-SI',
                        'SUBINDUSTRY-Home Furnishings': 'HOMFURNIS-SI',
                        'SUBINDUSTRY-Gas Utilities': 'GASUTIL-SI',
                        'SUBINDUSTRY-Insurance Brokers': 'INSURBROK-SI',
                        'SUBINDUSTRY-Leisure Products': 'LEISPROD-SI',
                        'SUBINDUSTRY-Micro Finance Institutions': 'MICROFININS-SI',
                        'SUBINDUSTRY-Oil & Gas Equipment & Services': 'ONGEQUSERV-SI',
                        'SUBINDUSTRY-Diversified Capital Markets': 'DIVERCAPMKT-SI',
                        'SECTOR-Consumer Discretionary': 'CONSDISC-S',
                        'SECTOR-Consumer Staples': 'CONSTAPL-S',
                        'SECTOR-Industrials': 'INDUSTRIAL-S',
                        'SECTOR-Materials': 'MATERIALS-S',
                        'SECTOR-Energy': 'ENERGY-S',
                        'SECTOR-Health Care': 'HLTHCARE-S',
                        'SECTOR-Information Technology': 'INFOTEC-S',
                        'SECTOR-Financials': 'FINANCIALS-S',
                        'SECTOR-Utilities': 'UTILITIES-S',
                        'SECTOR-Communication Services': 'COMMSERV-S',
                        'SECTOR-Real Estate': 'REALESTATE-S',
                        'SUBSECTOR-Automobiles & Components': 'AUTOCOMP-SS',
                        'SUBSECTOR-Consumer Discretionary Distribution & Retail': 'CONSDDRETA-SS',
                        'SUBSECTOR-Consumer Staples Distribution & Retail': 'CONSTAPDISRE-SS',
                        'SUBSECTOR-Food, Beverage & Tobacco': 'FOOBEVTOBA-SS',
                        'SUBSECTOR-Capital Goods': 'CAPIGOOD-SS',
                        'SUBSECTOR-Materials': 'MATERIAL-SS',
                        'SUBSECTOR-Consumer Durables & Apparel': 'CONSDURAPP-SS',
                        'SUBSECTOR-Energy': 'ENERGY-SS',
                        'SUBSECTOR-Household & Personal Products': 'HOUSPERS-SS',
                        'SUBSECTOR-Pharmaceuticals, Biotechnology & Life Sciences': 'PHARMBIOLIFS-SS',
                        'SUBSECTOR-Health Care Equipment & Services': 'HLTCAREQUSV-SS',
                        'SUBSECTOR-Technology Hardware & Equipment': 'TECHARDEQUI-SS',
                        'SUBSECTOR-Software & Services': 'SOFTSERV-SS',
                        'SUBSECTOR-Commercial & Professional Services': 'COMPRFSERV-SS',
                        'SUBSECTOR-Insurance': 'INSURANCE-SS',
                        'SUBSECTOR-Banks': 'BANKS-SS',
                        'SUBSECTOR-Financial Services': 'FINSERV-SS',
                        'SUBSECTOR-Utilities': 'UTILITIES-SS',
                        'SUBSECTOR-Telecommunication Services': 'TELECOSERV-SS',
                        'SUBSECTOR-Transportation': 'TRANSPORT-SS',
                        'SUBSECTOR-Consumer Services': 'CONSERV-SS',
                        'SUBSECTOR-Media & Entertainment': 'MEDIAENT-SS',
                        'SUBSECTOR-Real Estate Management & Development': 'RESMGMDEV-SS',
                        'SUBSECTOR-Equity Real Estate Investment Trusts (REITs)': 'EQREITS-SS'
                    }

# create a new table called irs_index_mapping
cur.execute('DROP TABLE IF EXISTS irs_index_mapping')
cur.execute('CREATE TABLE irs_index_mapping (IndexName TEXT, IndexMapping TEXT)')
conn.commit()

# insert the index_mapping into the irs_index_mapping table
for key, value in INDEX_MAPPING.items():
    cur.execute('INSERT INTO irs_index_mapping (IndexName, IndexMapping) VALUES (%s, %s)', (key, value))
    conn.commit()



# setting up Avg_category_mapping table

In [ ]:
# create a new table called avg_category_mapping
import pandas as pd
from utils.db_helper import DB_Helper

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

avg_category_mapping = pd.read_csv('C:\BravisaBackup\Avg_category mapping.csv')

cur.execute('DROP TABLE IF EXISTS avg_category_mapping')
# columns are:  scheme_name, scheme_category, date, btt_scheme_code, btt_scheme_category
cur.execute('CREATE TABLE avg_category_mapping (scheme_name TEXT, scheme_category TEXT, date DATE, btt_scheme_code TEXT, btt_scheme_category TEXT)')
conn.commit()

for index, row in avg_category_mapping.iterrows():
    cur.execute('INSERT INTO avg_category_mapping (scheme_name, scheme_category, date, btt_scheme_code, btt_scheme_category) VALUES (%s, %s, %s, %s, %s)', (row['scheme_name'], row['scheme_category'], row['date'], row['btt_scheme_code'], row['btt_scheme_category']))
    conn.commit()
    print(f"Inserted row {index}")

## setting ignore list for mf_category_mapping 

In [ ]:
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

# create a new table called ignore_scheme_master
cur.execute('DROP TABLE IF EXISTS ignore_scheme_master')
# columns are scheme_code
cur.execute('CREATE TABLE ignore_scheme_master (scheme_code NUMERIC)')
conn.commit()

# get all unique scheme codes from SchemeMaster table
unique_schemecodes_sql = """SELECT DISTINCT "SchemeCode" FROM public."SchemeMaster" """
unique_schemecodes = pd.read_sql_query(unique_schemecodes_sql, conn)
print(unique_schemecodes.shape)

# get all unique scheme codes from mf_category_mapping table
unique_schemecodes_sql = """SELECT DISTINCT "scheme_code" FROM public."mf_category_mapping" """
mf_category_mapping = pd.read_sql_query(unique_schemecodes_sql, conn)
print(mf_category_mapping.shape)

# remove all the schemes that are present in mf_category_mapping table
unique_schemecodes = unique_schemecodes[~unique_schemecodes['SchemeCode'].isin(mf_category_mapping['scheme_code'])]
print(unique_schemecodes.shape)

# insert the unique scheme codes into the ignore_scheme_master table
for index, row in unique_schemecodes.iterrows():
    cur.execute('INSERT INTO ignore_scheme_master (scheme_code) VALUES (%s)', (row['SchemeCode'],))
    conn.commit()
    print(f"Inserted row {index}")
    
    


(16897, 1)
(1315, 1)
(15582, 1)


C:\Users\dsram\AppData\Local\Temp\ipykernel_15096\3412820218.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  unique_schemecodes = pd.read_sql_query(unique_schemecodes_sql, conn)
C:\Users\dsram\AppData\Local\Temp\ipykernel_15096\3412820218.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mf_category_mapping = pd.read_sql_query(unique_schemecodes_sql, conn)


## Running misc reports

index close change and BTTDivisor

In [1]:
from utils.index_change import IndexCloseChange
from utils.BTTIndex import BTTIndex
import pandas as pd 
from utils.db_helper import DB_Helper
import warnings
warnings.filterwarnings("ignore")

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

start_date = '2020-08-01'
end_date = '2025-02-14'

def delete_index_change_values(date):
    sql = f'DELETE FROM public."nse_index_change" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted index change values for {date}")
    
def delete_BTTDivisor(date):
    sql = f'DELETE FROM public."BTTDivisor" WHERE "Date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted BTTDivisor values for {date}")
    

    
for date in pd.date_range(start=start_date, end=end_date):
    delete_index_change_values(date.date())
    delete_BTTDivisor(date.date())  
    IndexCloseChange().generating_daily_nse_index_changeValues_df(date.date(), conn, cur)
    BTTIndex().cal_BTT_divisor_Index(date.date(),conn,cur)
    



Deleted index change values for 2020-08-01
Deleted BTTDivisor values for 2020-08-01
NSE index data
NSE500 and NIFTY Index Daily data
NSE500 Change calculated Data
NIFTY Index Change calculated Data
Inserting the NSE500 Change df to the DB
Inserting the NIFTY index Change df to the DB


Getting BTT Divisor for date:  2020-08-01
Calculating PrevCLose for btt
Inserting btt divisor data


Getting BTT Index list for date:  2020-08-01


Insert BTT list for date:  2020-08-01


Getting BTT list for date:  2020-08-01


Insert BTT list for date: 
Deleted index change values for 2020-08-02
Deleted BTTDivisor values for 2020-08-02
NSE index data
NSE500 and NIFTY Index Daily data
NSE500 Change calculated Data
NIFTY Index Change calculated Data
Inserting the NSE500 Change df to the DB
Inserting the NIFTY index Change df to the DB


Getting BTT Divisor for date:  2020-08-02
Calculating PrevCLose for btt
Inserting btt divisor data


Getting BTT Index list for date:  2020-08-02


Insert BTT list for da

CombinedRS

In [ ]:
from utils.db_helper import DB_Helper
import pandas as pd
from datetime import datetime
from  reports.combined_rank import CombinedRank
import warnings
warnings.filterwarnings("ignore")

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

start_date = '2020-08-05'
end_date = '2025-02-14' 

def delete_combined_rank(date):
    sql = f'DELETE FROM "Reports"."CombinedRS" WHERE "GenDate" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted Combined Rank data for {date}")

date_range = pd.date_range(start=start_date, end=end_date)
for curr_date in date_range:
    delete_combined_rank(curr_date.date())
    CombinedRank().current_rank(curr_date, conn,cur)

perstock_offhighlow and perstock_change

In [ ]:
import pandas as pd
from datetime import datetime
from utils.db_helper import DB_Helper
from dash_process import perstock_offhighlow
from dash_process import perstock_change
import warnings
warnings.filterwarnings("ignore")

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

start_date = '2020-08-04'
end_date = '2025-02-14'

def delete_perstock_high(date):
    sql = f'DELETE FROM "dash_process"."stock_off_high" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted stock_off_high data for {date}")
    
def delete_perstocloff_low(date):
    sql = f'DELETE FROM "dash_process"."stock_off_low" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted stock_off_low data for {date}")
    
def delete_perstock_change(date):
    sql = f'DELETE FROM "dash_process"."stock_performance" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted stock_performance data for {date}")
    
    
date_range = pd.date_range(start=start_date, end=end_date)
for curr_date in date_range:
        delete_perstock_high(curr_date.date())
        delete_perstocloff_low(curr_date.date())
        delete_perstock_change(curr_date.date())
    
        perstock_offhighlow.main(curr_date)
        try:
            perstock_change.main(curr_date)
        except Exception as e:
            print(e, flush=True)

Deleted stock_off_high data for 2020-08-04
Deleted stock_off_low data for 2020-08-04
Deleted stock_performance data for 2020-08-04
Starting stock off-high/low process for date: 2020-08-04 00:00:00

Getting PRS for current date
Calculating off-high for stocks
Calculating off-low for stocks
Inserting off-high data into DB
Inserting off-low data into DB

Getting OHLC for current date: 2020-08-04 00:00:00
Getting OHLC back
Getting BTTList
Merging current day's OHLC with BTT List
Merging backdate OHLC with BTT List
Calculating change value for stocks
Inserting into the DB
Deleted stock_off_high data for 2020-08-05
Deleted stock_off_low data for 2020-08-05
Deleted stock_performance data for 2020-08-05
Starting stock off-high/low process for date: 2020-08-05 00:00:00

Getting PRS for current date
Calculating off-high for stocks
Calculating off-low for stocks
Inserting off-high data into DB
Inserting off-low data into DB

Getting OHLC for current date: 2020-08-05 00:00:00
